In [2]:
#Recomendation system 

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

df = pd.read_csv("datasiap.csv")
df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date_Mon_Year,Month,TotalAmount,Hour,Minute
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,01-Dec-2010,Dec,15.30,8,26
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,01-Dec-2010,Dec,20.34,8,26
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,01-Dec-2010,Dec,22.00,8,26
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,01-Dec-2010,Dec,20.34,8,26
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,01-Dec-2010,Dec,20.34,8,26


In [6]:
del df['Unnamed: 0']

In [7]:
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['desc_clean'] = df['Description'].apply(clean_text)

In [8]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date_Mon_Year,Month,TotalAmount,Hour,Minute,desc_clean
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,01-Dec-2010,Dec,15.30,8,26,white hanging heart tlight holder
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,01-Dec-2010,Dec,20.34,8,26,white metal lantern
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,01-Dec-2010,Dec,22.00,8,26,cream cupid hearts coat hanger
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,01-Dec-2010,Dec,20.34,8,26,knitted union flag hot water bottle
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,01-Dec-2010,Dec,20.34,8,26,red woolly hottie white heart
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392687,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,09-Dec-2011,Dec,10.20,12,50,pack 20 spaceboy napkins
392688,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,09-Dec-2011,Dec,12.60,12,50,childrens apron dolly girl
392689,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,09-Dec-2011,Dec,16.60,12,50,childrens cutlery dolly girl
392690,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,09-Dec-2011,Dec,16.60,12,50,childrens cutlery circus parade


In [9]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date_Mon_Year,Month,TotalAmount,Hour,Minute,desc_clean
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,01-Dec-2010,Dec,15.30,8,26,white hanging heart tlight holder
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,01-Dec-2010,Dec,20.34,8,26,white metal lantern
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,01-Dec-2010,Dec,22.00,8,26,cream cupid hearts coat hanger
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,01-Dec-2010,Dec,20.34,8,26,knitted union flag hot water bottle
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,01-Dec-2010,Dec,20.34,8,26,red woolly hottie white heart
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392687,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,09-Dec-2011,Dec,10.20,12,50,pack 20 spaceboy napkins
392688,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,09-Dec-2011,Dec,12.60,12,50,childrens apron dolly girl
392689,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,09-Dec-2011,Dec,16.60,12,50,childrens cutlery dolly girl
392690,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,09-Dec-2011,Dec,16.60,12,50,childrens cutlery circus parade


In [10]:
df=df['Description']
df

0          WHITE HANGING HEART T-LIGHT HOLDER
1                         WHITE METAL LANTERN
2              CREAM CUPID HEARTS COAT HANGER
3         KNITTED UNION FLAG HOT WATER BOTTLE
4              RED WOOLLY HOTTIE WHITE HEART.
                         ...                 
392687            PACK OF 20 SPACEBOY NAPKINS
392688           CHILDREN'S APRON DOLLY GIRL 
392689          CHILDRENS CUTLERY DOLLY GIRL 
392690        CHILDRENS CUTLERY CIRCUS PARADE
392691          BAKING SET 9 PIECE RETROSPOT 
Name: Description, Length: 392692, dtype: object

In [11]:
df1 = pd.DataFrame(df)

In [12]:
df1

,Description
0,WHITE HANGING HEART T-LIGHT HOLDER
1,WHITE METAL LANTERN
2,CREAM CUPID HEARTS COAT HANGER
3,KNITTED UNION FLAG HOT WATER BOTTLE
4,RED WOOLLY HOTTIE WHITE HEART.
...,...
392687,PACK OF 20 SPACEBOY NAPKINS
392688,CHILDREN'S APRON DOLLY GIRL
392689,CHILDRENS CUTLERY DOLLY GIRL
392690,CHILDRENS CUTLERY CIRCUS PARADE


In [13]:
dfj=df1['Description'].unique()

In [14]:
df = pd.DataFrame(dfj)

In [15]:
df

,0
0,WHITE HANGING HEART T-LIGHT HOLDER
1,WHITE METAL LANTERN
2,CREAM CUPID HEARTS COAT HANGER
3,KNITTED UNION FLAG HOT WATER BOTTLE
4,RED WOOLLY HOTTIE WHITE HEART.
...,...
3872,"LETTER ""W"" BLING KEY RING"
3873,"LETTER ""Z"" BLING KEY RING"
3874,PINK CRYSTAL SKULL PHONE CHARM
3875,CREAM HANGING HEART T-LIGHT HOLDER


In [16]:
df[0]

0        WHITE HANGING HEART T-LIGHT HOLDER
1                       WHITE METAL LANTERN
2            CREAM CUPID HEARTS COAT HANGER
3       KNITTED UNION FLAG HOT WATER BOTTLE
4            RED WOOLLY HOTTIE WHITE HEART.
                       ...                 
3872              LETTER "W" BLING KEY RING
3873              LETTER "Z" BLING KEY RING
3874         PINK CRYSTAL SKULL PHONE CHARM
3875     CREAM HANGING HEART T-LIGHT HOLDER
3876            PAPER CRAFT , LITTLE BIRDIE
Name: 0, Length: 3877, dtype: object

In [17]:
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['desc_clean'] = df[0].apply(clean_text)

In [18]:
df

,0,desc_clean
0,WHITE HANGING HEART T-LIGHT HOLDER,white hanging heart tlight holder
1,WHITE METAL LANTERN,white metal lantern
2,CREAM CUPID HEARTS COAT HANGER,cream cupid hearts coat hanger
3,KNITTED UNION FLAG HOT WATER BOTTLE,knitted union flag hot water bottle
4,RED WOOLLY HOTTIE WHITE HEART.,red woolly hottie white heart
...,...,...
3872,"LETTER ""W"" BLING KEY RING",letter w bling key ring
3873,"LETTER ""Z"" BLING KEY RING",letter z bling key ring
3874,PINK CRYSTAL SKULL PHONE CHARM,pink crystal skull phone charm
3875,CREAM HANGING HEART T-LIGHT HOLDER,cream hanging heart tlight holder


In [19]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
cv = CountVectorizer()

count_matrix = cv.fit_transform(df['desc_clean'])

In [21]:
print(count_matrix)

  (0, 1972)	1
  (0, 868)	1
  (0, 879)	1
  (0, 1854)	1
  (0, 903)	1
  (1, 1972)	1
  (1, 1132)	1
  (1, 1014)	1
  (2, 492)	1
  (2, 513)	1
  (2, 880)	1
  (2, 433)	1
  (2, 866)	1
  (3, 998)	1
  (3, 1924)	1
  (3, 704)	1
  (3, 917)	1
  (3, 1962)	1
  (3, 227)	1
  (4, 1972)	1
  (4, 879)	1
  (4, 1493)	1
  (4, 1995)	1
  (4, 918)	1
  (5, 1605)	1
  :	:
  (3871, 1037)	1
  (3871, 980)	1
  (3871, 199)	1
  (3872, 1520)	1
  (3872, 1037)	1
  (3872, 980)	1
  (3872, 199)	1
  (3873, 1520)	1
  (3873, 1037)	1
  (3873, 980)	1
  (3873, 199)	1
  (3874, 1372)	1
  (3874, 1661)	1
  (3874, 504)	1
  (3874, 365)	1
  (3874, 1357)	1
  (3875, 868)	1
  (3875, 879)	1
  (3875, 1854)	1
  (3875, 903)	1
  (3875, 492)	1
  (3876, 1303)	1
  (3876, 485)	1
  (3876, 1058)	1
  (3876, 187)	1


In [22]:
print(count_matrix.toarray())


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [23]:
cos_sim = cosine_similarity(count_matrix)

In [24]:
df

,0,desc_clean
0,WHITE HANGING HEART T-LIGHT HOLDER,white hanging heart tlight holder
1,WHITE METAL LANTERN,white metal lantern
2,CREAM CUPID HEARTS COAT HANGER,cream cupid hearts coat hanger
3,KNITTED UNION FLAG HOT WATER BOTTLE,knitted union flag hot water bottle
4,RED WOOLLY HOTTIE WHITE HEART.,red woolly hottie white heart
...,...,...
3872,"LETTER ""W"" BLING KEY RING",letter w bling key ring
3873,"LETTER ""Z"" BLING KEY RING",letter z bling key ring
3874,PINK CRYSTAL SKULL PHONE CHARM,pink crystal skull phone charm
3875,CREAM HANGING HEART T-LIGHT HOLDER,cream hanging heart tlight holder


In [25]:
df.columns = ['title', 'desc_clean']

In [26]:
product_user_likes = 'PINK CRYSTAL SKULL PHONE CHARM'

In [27]:
df

,title,desc_clean
0,WHITE HANGING HEART T-LIGHT HOLDER,white hanging heart tlight holder
1,WHITE METAL LANTERN,white metal lantern
2,CREAM CUPID HEARTS COAT HANGER,cream cupid hearts coat hanger
3,KNITTED UNION FLAG HOT WATER BOTTLE,knitted union flag hot water bottle
4,RED WOOLLY HOTTIE WHITE HEART.,red woolly hottie white heart
...,...,...
3872,"LETTER ""W"" BLING KEY RING",letter w bling key ring
3873,"LETTER ""Z"" BLING KEY RING",letter z bling key ring
3874,PINK CRYSTAL SKULL PHONE CHARM,pink crystal skull phone charm
3875,CREAM HANGING HEART T-LIGHT HOLDER,cream hanging heart tlight holder


In [28]:
df['index'] = df.index

In [29]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]

def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

In [30]:
movie_index = get_index_from_title(product_user_likes)

In [31]:
movie_index

3874

In [35]:
similar_movies = list(enumerate(cos_sim[movie_index]))

In [36]:
similar_movies

[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.0),
 (18, 0.0),
 (19, 0.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.22360679774997896),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0),
 (32, 0.0),
 (33, 0.0),
 (34, 0.0),
 (35, 0.0),
 (36, 0.0),
 (37, 0.0),
 (38, 0.0),
 (39, 0.0),
 (40, 0.0),
 (41, 0.0),
 (42, 0.0),
 (43, 0.0),
 (44, 0.0),
 (45, 0.0),
 (46, 0.0),
 (47, 0.0),
 (48, 0.0),
 (49, 0.0),
 (50, 0.0),
 (51, 0.0),
 (52, 0.0),
 (53, 0.0),
 (54, 0.0),
 (55, 0.0),
 (56, 0.0),
 (57, 0.0),
 (58, 0.0),
 (59, 0.22360679774997896),
 (60, 0.0),
 (61, 0.0),
 (62, 0.0),
 (63, 0.0),
 (64, 0.0),
 (65, 0.0),
 (66, 0.0),
 (67, 0.0),
 (68, 0.0),
 (69, 0.0),
 (70, 0.0),
 (71, 0.18257418583505539),
 (72, 0.0),
 (73, 0.0),
 (74, 0.0),
 (75, 0.0),
 (76, 0.0),
 (77, 0.19999999999999998),
 (78, 0.0)

In [37]:
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)

In [38]:
sorted_similar_movies

[(3874, 0.9999999999999999),
 (2825, 0.7999999999999999),
 (1573, 0.6708203932499369),
 (3167, 0.6708203932499369),
 (3166, 0.6),
 (3599, 0.6),
 (3841, 0.6),
 (2484, 0.4472135954999579),
 (3641, 0.4472135954999579),
 (1420, 0.39999999999999997),
 (1564, 0.39999999999999997),
 (1624, 0.39999999999999997),
 (2125, 0.39999999999999997),
 (2540, 0.39999999999999997),
 (2206, 0.36514837167011077),
 (89, 0.25819888974716115),
 (363, 0.25819888974716115),
 (369, 0.25819888974716115),
 (487, 0.25819888974716115),
 (599, 0.25819888974716115),
 (742, 0.25819888974716115),
 (878, 0.25819888974716115),
 (998, 0.25819888974716115),
 (1104, 0.25819888974716115),
 (1105, 0.25819888974716115),
 (1113, 0.25819888974716115),
 (1214, 0.25819888974716115),
 (1298, 0.25819888974716115),
 (1450, 0.25819888974716115),
 (1469, 0.25819888974716115),
 (1597, 0.25819888974716115),
 (1614, 0.25819888974716115),
 (1753, 0.25819888974716115),
 (1771, 0.25819888974716115),
 (1822, 0.25819888974716115),
 (1944, 0.258

In [39]:
for movie in sorted_similar_movies:
    print (get_title_from_index(movie[0]))

PINK CRYSTAL SKULL PHONE CHARM
PINK CRYSTAL GUITAR PHONE CHARM
CRYSTAL FROG PHONE CHARM
CRYSTAL STILETTO PHONE CHARM
CLEAR CRYSTAL STAR PHONE CHARM
CRYSTAL SEA HORSE PHONE CHARM
CRYSTAL CZECH CROSS PHONE CHARM
PINK CRYSTAL+GLASS BRACELET
PINK DAISY BAG CHARM
BLUE LEAVES AND BEADS PHONE CHARM
BEADED CRYSTAL HEART PINK LARGE
BEADED CRYSTAL HEART PINK SMALL
PINK GLASS TASSLE BAG CHARM 
BEADED CRYSTAL HEART PINK ON STICK
PINK BERTIE GLASS BEAD BAG CHARM
SKULL SHOULDER BAG
PINK  POLKADOT PLATE 
6 RIBBONS RUSTIC CHARM
PINK SWEETHEART BRACELET
PINK PARTY BAGS
POLKADOT MUG PINK 
EDWARDIAN PARASOL PINK
WATERING CAN PINK BUNNY
PINK  POLKADOT CUP
SKULL AND CROSSBONES  GARLAND 
PINK POLKADOT BOWL
PINK POLKADOT WRAP 
WRAP  PINK FLOCK
PINK DOG BOWL
SET/4 SKULL BADGES
STRING OF 8 BUTTERFLIES,PINK
PINK PADDED MOBILE
PINK PARTY SUNGLASSES
PINK ROSE WASHBAG
PINK GLASS CANDLEHOLDER
PINK CAT BOWL
PAINTED PINK RABBIT 
NINJA RABBIT PINK
PINK BUTTERFLY WASHBAG
PINK DOG CANNISTER
PINK FLY SWAT
PINK BABY BUNTI

CHILLI LIGHTS
WOODEN OWLS LIGHT GARLAND 
FAIRY TALE COTTAGE NIGHTLIGHT
HOME SMALL WOOD LETTERS
GINGHAM HEART  DOORSTOP RED
FIVE HEART HANGING DECORATION
ASSORTED BOTTLE TOP  MAGNETS 
FRIDGE MAGNETS US DINER ASSORTED
HOMEMADE JAM SCENTED CANDLES
FRIDGE MAGNETS LES ENFANTS ASSORTED
ROSE CARAVAN DOORSTOP
5 HOOK HANGER MAGIC TOADSTOOL
CHRISTMAS LIGHTS 10 REINDEER
VINTAGE UNION JACK CUSHION COVER
SET OF 3 COLOURED  FLYING DUCKS
SET OF 3 GOLD FLYING DUCKS
RED RETROSPOT UMBRELLA
BLACK/BLUE POLKADOT UMBRELLA
RED DINER WALL CLOCK
BLUE DINER WALL CLOCK
IVORY DINER WALL CLOCK
LARGE HEART MEASURING SPOONS
SMALL HEART MEASURING SPOONS
JAM JAR WITH GREEN LID
ROSE COTTAGE KEEPSAKE BOX 
HANGING HEART ZINC T-LIGHT HOLDER
PAPER CHAIN KIT VINTAGE CHRISTMAS
DISCO BALL CHRISTMAS DECORATION
SMALL POPCORN HOLDER
LARGE POPCORN HOLDER 
SET/20 RED RETROSPOT PAPER NAPKINS 
SET/6 RED SPOTTY PAPER CUPS
POLKADOT RAIN HAT 
DELUXE SEWING KIT 
RETROSPOT HEART HOT WATER BOTTLE
ENGLISH ROSE HOT WATER BOTTLE
PHOTO CUBE
3

RECIPE BOX RETROSPOT 
RECIPE BOX PANTRY YELLOW DESIGN
RECIPE BOX BLUE SKETCHBOOK DESIGN
20 DOLLY PEGS RETROSPOT
NO SINGING METAL SIGN
BISCUIT TIN VINTAGE RED
EMERGENCY FIRST AID TIN 
POLKADOT PEN
DECORATIVE PLANT POT WITH FRIEZE
FRENCH BLUE METAL DOOR SIGN 5
CHEST OF DRAWERS GINGHAM HEART 
RED RETROSPOT CAKE STAND
JOY WOODEN BLOCK LETTERS
NOEL WOODEN BLOCK LETTERS 
PEACE WOODEN BLOCK LETTERS
OFFICE MUG WARMER BLACK+SILVER 
SET OF 2 CHRISTMAS DECOUPAGE CANDLE
COSMETIC BAG VINTAGE ROSE PAISLEY
SILVER PLATE CANDLE BOWL SMALL
SMALL REGAL  SILVER CANDLEPOT 
ANTIQUE TALL SWIRLGLASS TRINKET POT
ANTIQUE GLASS PEDESTAL BOWL
GUMBALL MAGAZINE RACK
RED FLOWER CROCHET FOOD COVER
CHERRY CROCHET FOOD COVER
STRAWBERRY RAFFIA FOOD COVER
POLKA DOT RAFFIA FOOD COVER
DOORMAT SPOTTY HOME SWEET HOME
T-LIGHT GLASS FLUTED ANTIQUE
SILVER  CANDLEPOT JARDIN 
PAPER BUNTING COLOURED LACE
ANTIQUE GLASS HEART DECORATION 
SET OF 6 KASHMIR FOLKART BAUBLES
CHRISTMAS RETROSPOT ANGEL WOOD
CHRISTMAS RETROSPOT STAR WOOD
HE

PHOTO FRAME 3 CLASSIC HANGING
3D TRADITIONAL CHRISTMAS STICKERS
TRADITIONAL MODELLING CLAY
SEASIDE FLYING DISC
PEG BAG APPLES DESIGN
GREY FLORAL FELTCRAFT SHOULDER BAG
CHARLIE LOLA BLUE HOT WATER BOTTLE 
SET/3 RED GINGHAM ROSE STORAGE BOX
GLASS HEART T-LIGHT HOLDER 
TEA TIME PARTY BUNTING
WOODEN UNION JACK BUNTING
PAPER BUNTING RETROSPOT
SMALL WHITE RETROSPOT MUG IN BOX 
SMALL RED RETROSPOT MUG IN BOX 
SET 3 RETROSPOT TEA,COFFEE,SUGAR
12 PENCILS TALL TUBE RED RETROSPOT
RED RETROSPOT PUDDING BOWL
RED RETROSPOT TRADITIONAL TEAPOT 
DAIRY MAID  PUDDING BOWL
ASSORTED CAKES FRIDGE MAGNETS
FOOD CONTAINER SET 3 LOVE HEART 
DO NOT TOUCH MY STUFF DOOR HANGER 
MOODY GIRL DOOR HANGER 
KEEP OUT BOYS DOOR HANGER 
MOODY BOY  DOOR HANGER 
FILIGRIS HEART WITH BUTTERFLY
HEART DECORATION PAINTED ZINC 
DECORATION HEN ON NEST, HANGING
HEN HOUSE DECORATION
PACK OF 12 STICKY BUNNIES
MINI WOODEN HAPPY BIRTHDAY GARLAND
BLUE HAPPY BIRTHDAY BUNTING
WOOD STAMP SET BEST WISHES
WOOD STAMP SET HAPPY BIRTHDAY
WOOD ST

CARD SUKI BIRTHDAY
WRAP SUKI AND FRIENDS
BIRDHOUSE DECORATION MAGIC GARDEN
BIRDCAGE DECORATION TEALIGHT HOLDER
BROWN  PIRATE TREASURE CHEST 
BLUE PUDDING SPOON
SUMMER FUN DESIGN SHOWER CAP
STRIPY DESIGN SHOWER CAP
SET/4 DAISY MIRROR MAGNETS
SET/4 BIRD MIRROR MAGNETS 
SET/4 BUTTERFLY MIRROR MAGNETS
FRENCH STYLE EMBOSSED HEART CABINET
GREEN SWEETHEART BRACELET
CARNIVAL BRACELET
4 PEAR BOTANICAL DINNER CANDLES
FUNKY MONKEY MUG
QUEEN OF SKIES LUGGAGE TAG
ECONOMY LUGGAGE TAG
HEART CALCULATOR
BISCUITS SMALL BOWL LIGHT BLUE
PARTY PIZZA DISH GREEN POLKADOT
PARTY PIZZA DISH BLUE POLKADOT
ASSORTED CHEESE FRIDGE MAGNETS
WHITE STITCHED CUSHION COVER
PORCELAIN BUTTERFLY OIL BURNER
BREAD BIN, DINER STYLE, MINT
BREAD BIN, DINER STYLE, IVORY
SET OF 6 VINTAGE NOTELETS KIT
CARD GINGHAM ROSE 
SET OF 12  VINTAGE POSTCARD SET
LADLE LOVE HEART RED 
36 DOILIES DOLLY GIRL
ROSE 1 WICK MORRIS BOXED CANDLE
ASSORTED TUTTI FRUTTI  FOB NOTEBOOK
DOLLY HONEYCOMB GARLAND
ROSE FOLKART HEART DECORATIONS
GLITTER CHRISTMA

BLUE SHARK HELICOPTER
GREEN WIRE STANDING CANDLE HOLDER
ROMANTIC IMAGES SCRAP BOOK SET
MINI JIGSAW PURDEY
LETTER SHAPE PENCIL SHARPENER
BLACK STITCHED WALL CLOCK
DOOR HANGER  MUM + DADS ROOM
QUEEN OF THE SKIES HOLIDAY PURSE 
GOLD SCROLL GLASS T-LIGHT HOLDER
GLASS JAR MARMALADE 
FLOWER GLASS GARLD NECKL36"AMETHYST
PURPLE BOUDICCA LARGE BRACELET
RED BOUDICCA LARGE BRACELET
ANT COPPER TURQ BOUDICCA BRACELET
TURQ+RED BOUDICCA LARGE BRACELET
BATHROOM SCALES, TROPICAL BEACH
CHARLIE & LOLA WASTEPAPER BIN BLUE
BLACK CHRISTMAS FLOCK DROPLET 
WHITE CHRISTMAS FLOCK DROPLET 
FLOWERS TILE HOOK
DANISH ROSE BEDSIDE CABINET
CAKE STAND VICTORIAN FILIGREE LARGE
WHITE MURANO TWIST BRACELET
DOORSTOP RACING CAR DESIGN
BLACK SILOUETTE CANDLE PLATE
4 VANILLA BOTANICAL CANDLES
HEN PARTY CORDON BARRIER TAPE
VINTAGE UNION JACK SHOPPING BAG
FRENCH KITCHEN SIGN BLUE METAL
EMPIRE GIFT WRAP
CANDLE PLATE LACE WHITE
ANT SILVER FUSCHIA BOUDICCA RING
GOLD M PEARL  ORBIT NECKLACE
CHUNKY SILVER NECKLACE PASTEL FLOWE
IVOR

RUBY GLASS CLUSTER BRACELET
MIDNIGHT BLUE GLASS/SILVER BRACELET
PAINTED SEA SHELL METAL WINDCHIME
FOLDING SHOE TIDY
MADRAS NOTEBOOK MEDIUM
SQUARE METAL CANDLEHOLDER BASE
ASSORTED TUTTI FRUTTI KEYRING BALL
M/COLOUR POM-POM CURTAIN
BLACK CHERRY LIGHTS
CRACKED GLAZE EARRINGS BROWN
NEW BAROQUE SMALL NECKLACE BLACK
SWEETHEART CARRY-ALL BASKET
LILAC GAUZE BUTTERFLY LAMPSHADE
GIANT MEDINA STAMPED METAL BOWL 
PURPLE AMETHYST NECKLACE W TASSEL
BLUE/NAT SHELL NECKLACE W PENDANT
SILVER/NAT SHELL NECKLACE W PENDANT
SILVER M.O.P ORBIT DROP EARRINGS
FILIGREE DIAMANTE EARRINGS
JADE DROP EARRINGS W FILIGREE
GRASS HOPPER WOODEN WALL CLOCK 
IVORY STRING CURTAIN WITH POLE 
MUSICAL ZINC HEART DECORATION 
RETROSPOT WOODEN HEART DECORATION
HANGING METAL CHICKEN DECORATION
PANTRY SCRUBBING BRUSH
HANGING HEART WITH BELL
SET OF 4 PANTRY JELLY MOULDS
TRAVEL CARD WALLET FLOWER MEADOW
TRAVEL CARD WALLET PANTRY
TRAVEL CARD WALLET SKULLS
TRAVEL CARD WALLET VINTAGE ROSE 
TRAVEL CARD WALLET TRANSPORT
TRAVEL CARD WALL

PAPER LANTERN 5 POINT STAR MOON 30
PAPER LANTERN 6 POINT SNOW STAR
BUNTING , SPOTTY 
POLYESTER FILLER PAD 60x40cm
BLUE DISCO HANDBAG
LUNCH BAG DOILEY PATTERN 
JUMBO BAG PEARS
JUMBO BAG DOILEY PATTERNS
MONTANA DIAMOND CLUSTER EARRINGS
MARIE ANTOINETTE TRINKET BOX GOLD
MARIE ANTOINETTE TRINKET BOX SILVER
JUMBO BAG VINTAGE LEAF
PAPER LANTERN 9 POINT HOLLY STAR L
PAPER LANTERN 9 POINT DELUXE STAR
PAPER LANTERN 5 POINT SEQUIN STAR
PAPER LANTERN 9 POINT HOLLY STAR S
PAPER LANTERN 9 POINT HOLLY STAR 23
PAPER LANTERN 5 POINT STUDDED STAR
FOOD COVER WITH BEADS , SET 2 SIZES
RASTA IN BATH W SPLIFF ASHTRAY
FLOWER GARLAND NECKLACE RED
IVORY SHELL HEART EARRINGS
FLOWER GLASS GARLAND NECKL.36"BLUE
FOLDING BUTTERFLY MIRROR IVORY 
BUNDLE OF 3 SCHOOL EXERCISE BOOKS  
BUNDLE OF 3 RETRO NOTE BOOKS
GREEN GOOSE FEATHER TREE 60CM
WHITE GOOSE FEATHER TREE 60CM 
BUNDLE OF 3 RETRO EXERCISE BOOKS
REGENCY SUGAR BOWL GREEN
REGENCY TEAPOT ROSES 
REGENCY TEA PLATE GREEN 
REGENCY TEA PLATE ROSES 
GREEN METAL BOX ARM

 DOLLY GIRL BEAKER
SPACEBOY BEAKER
DOORKNOB CERAMIC IVORY
ORIENTAL RED CUSHION COVER 
MONEY BOX FIRST AID DESIGN
VINTAGE CHRISTMAS CAKE FRILL
MISELTOE HEART WREATH CREAM
GOLD M.O.P ORBIT BRACELET
SET 8 CANDLES VINTAGE DOILEY
SET OF 12 T-LIGHTS VINTAGE DOILEY
VINTAGE CHRISTMAS PAPER GIFT BAG
GIFT BAG LARGE VINTAGE CHRISTMAS
6 GIFT TAGS VINTAGE CHRISTMAS 
6 GIFT TAGS 50'S CHRISTMAS 
RED SPOT PAPER GIFT BAG
 RED SPOT GIFT BAG LARGE
SET OF 12 FORK CANDLES
BLUE PAINTED KASHMIRI CHAIR
ROLL WRAP VINTAGE CHRISTMAS
ROLL WRAP VINTAGE SPOT 
GIFT BAG LARGE SPOT 
ROLL WRAP 50'S RED CHRISTMAS 
ROLL WRAP 50'S CHRISTMAS
JUMBO BAG VINTAGE DOILEY 
ADULT APRON APPLE DELIGHT
WRAP  VINTAGE DOILEY 
SET OF 4 KNICK KNACK TINS DOILY 
RUSTIC STRAWBERRY JAMPOT LARGE 
PURPLE ENAMEL+GLASS HAIR COMB
JUMBO BAG VINTAGE DOILY 
LUNCH BAG VINTAGE DOILY 
SET 8 CANDLES VINTAGE DOILY
STORAGE TIN VINTAGE DOILY 
IVORY WICKER HEART LARGE
DECORATIVE WICKER HEART LARGE
ASSORTED COLOURED CIRCLE MOBILE 
MIDNIGHT BLUE VINTAGE EARR

TRADITIONAL PICK UP STICKS GAME 
SET OF 3 MINI HANGING PORTRAITS
TRADITIONAL NAUGHTS & CROSSES
TRADTIONAL ALPHABET STAMP SET
SET 4 PICNIC CUTLERY BLUEBERRY
THREE MINI HANGING FRAMES
WOODLAND SMALL BLUE FELT HEART
VINTAGE DOILY JUMBO BAG RED 
LUNCH BAG PAISLEY PARK  
JUMBO BAG PAISLEY PARK
UBO-LIGHT TRIOBASE BLUE
SET OF 36 VINTAGE CHRISTMAS DOILIES
SET OF 36 SPACEBOY PAPER DOILIES
SET OF 36 DOLLY GIRL PAPER DOILIES
STRAWBERRY CERAMIC TRINKET POT
DECORATIVE VINTAGE COFFEE  BOX
SET OF 36 PANTRY PAPER DOILIES
SET 4 PICNIC CUTLERY FONDANT
SILVER RECORD COVER FRAME
CLASSIC CHROME BICYCLE BELL 
SET 4 PICNIC CUTLERY CHERRY 
SET OF 3 PANTRY WOODEN SPOONS
PANTRY KITCHEN THERMOMETER 
LIGHT DECORATION BATTERY OPERATED
KEEP CALM BIRTHDAY WRAP
AMBER GLASS/SHELL/PEARL NECKLACE
5 STRAND GLASS NECKLACE AMETHYST
RUBY GLASS CLUSTER NECKLACE
CLASSIC DIAMANTE NECKLACE JET
DIAMANTE NECKLACE BLACK 
WRAP KEEP CALM BIRTHDAY
LUNCH BAG RED VINTAGE DOILY
BLACK SIL'T SQU CANDLE PLATE 
ANT SILVER TURQUOISE BOUDICCA

In [40]:
i=0
for element in sorted_similar_movies:
    print (get_title_from_index(element[0]))
    i =i+1
    if i > 10:
        break

PINK CRYSTAL SKULL PHONE CHARM
PINK CRYSTAL GUITAR PHONE CHARM
CRYSTAL FROG PHONE CHARM
CRYSTAL STILETTO PHONE CHARM
CLEAR CRYSTAL STAR PHONE CHARM
CRYSTAL SEA HORSE PHONE CHARM
CRYSTAL CZECH CROSS PHONE CHARM
PINK CRYSTAL+GLASS BRACELET
PINK DAISY BAG CHARM
BLUE LEAVES AND BEADS PHONE CHARM
BEADED CRYSTAL HEART PINK LARGE


In [41]:
# Fungsi recomendation

In [42]:
df=df[['title','desc_clean']]

In [43]:
df

,title,desc_clean
0,WHITE HANGING HEART T-LIGHT HOLDER,white hanging heart tlight holder
1,WHITE METAL LANTERN,white metal lantern
2,CREAM CUPID HEARTS COAT HANGER,cream cupid hearts coat hanger
3,KNITTED UNION FLAG HOT WATER BOTTLE,knitted union flag hot water bottle
4,RED WOOLLY HOTTIE WHITE HEART.,red woolly hottie white heart
...,...,...
3872,"LETTER ""W"" BLING KEY RING",letter w bling key ring
3873,"LETTER ""Z"" BLING KEY RING",letter z bling key ring
3874,PINK CRYSTAL SKULL PHONE CHARM,pink crystal skull phone charm
3875,CREAM HANGING HEART T-LIGHT HOLDER,cream hanging heart tlight holder


In [44]:
df.set_index('title', inplace=True)

In [45]:
indices = pd.Series(df.index)
indices[:50]

0      WHITE HANGING HEART T-LIGHT HOLDER
1                     WHITE METAL LANTERN
2          CREAM CUPID HEARTS COAT HANGER
3     KNITTED UNION FLAG HOT WATER BOTTLE
4          RED WOOLLY HOTTIE WHITE HEART.
5            SET 7 BABUSHKA NESTING BOXES
6       GLASS STAR FROSTED T-LIGHT HOLDER
7                  HAND WARMER UNION JACK
8               HAND WARMER RED POLKA DOT
9           ASSORTED COLOUR BIRD ORNAMENT
10             POPPY'S PLAYHOUSE BEDROOM 
11              POPPY'S PLAYHOUSE KITCHEN
12      FELTCRAFT PRINCESS CHARLOTTE DOLL
13                IVORY KNITTED MUG COSY 
14     BOX OF 6 ASSORTED COLOUR TEASPOONS
15          BOX OF VINTAGE JIGSAW BLOCKS 
16         BOX OF VINTAGE ALPHABET BLOCKS
17               HOME BUILDING BLOCK WORD
18               LOVE BUILDING BLOCK WORD
19            RECIPE BOX WITH METAL HEART
20                    DOORMAT NEW ENGLAND
21               JAM MAKING SET WITH JARS
22            RED COAT RACK PARIS FASHION
23         YELLOW COAT RACK PARIS 

In [46]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_product = []
    
    # Mengambil nama hotel berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 10 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    for i in top_10_indexes:
        recommended_product.append(list(df.index)[i])
        
    return recommended_product

In [47]:
recommendations('WHITE HANGING HEART T-LIGHT HOLDER')

['HANGING HEART ZINC T-LIGHT HOLDER',
 'CREAM HANGING HEART T-LIGHT HOLDER',
 'HANGING HEART JAR T-LIGHT HOLDER',
 'PINK HANGING HEART T-LIGHT HOLDER',
 'RED HANGING HEART T-LIGHT HOLDER',
 'HEART T-LIGHT HOLDER',
 'HEART T-LIGHT HOLDER ',
 'GLASS HEART T-LIGHT HOLDER ',
 'HANGING  BUTTERFLY T-LIGHT HOLDER',
 'HEART TRELLISTRIPLE T-LIGHT HOLDER']